In [21]:
!pip install vecstack

In [0]:
from vecstack import stacking
import pandas as pd
import numpy as np
from sklearn.metrics import accuracy_score #works
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import cross_val_score
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.neural_network import MLPClassifier
#from sklearn.ensemble import RandomForestClassifier
from imblearn.over_sampling import SMOTE 
from sklearn.svm import SVC
from collections import Counter #for Smote, 

import time
from sklearn.svm import LinearSVC

from sklearn.preprocessing import LabelEncoder
pd.set_option('display.max_columns',None)#displaying long list of columns
pd.set_option('display.max_rows', None)#displaying long list of rows
pd.set_option('display.width', 1000)#width of window

import warnings
warnings.filterwarnings("ignore")


In [23]:
from google.colab import drive
drive.mount('/gdrive')
#Change current working directory to gdrive
%cd /gdrive


trainfile = r'/gdrive/My Drive/assignment3/RevisedHomesiteTrain.csv'
train_data = pd.read_csv(trainfile)

testfile = r'/gdrive/My Drive/assignment3/RevisedHomesiteTest.csv'
test_data = pd.read_csv(testfile)

print(train_data.shape)

test_data=test_data.drop("GeographicField64",axis=1) 
print(test_data.shape)

X_train = train_data.iloc[:, :-1]
y_train = train_data.iloc[:, -1]
X_test = test_data

Drive already mounted at /gdrive; to attempt to forcibly remount, call drive.mount("/gdrive", force_remount=True).
/gdrive
(65000, 596)
(173836, 595)


In [34]:
#CONSTRUCT DEFAULT DECISION TREE
clf = DecisionTreeClassifier()
clf.fit(X_train,y_train)
clf_predict = clf.predict(X_test)
pred=pd.DataFrame(clf_predict,columns=["QuoteConversion_Flag"])

#Write into a file with actual prediction and corresponding probability
ID = test_data["QuoteNumber"]
pd.concat([ID,pred],axis=1).to_csv("/gdrive/My Drive/assignment3/DT.csv", index = None)

res=pd.read_csv('/gdrive/My Drive/assignment3/DT.csv')
res.head()

,QuoteNumber,QuoteConversion_Flag
0,3,0
1,5,1
2,7,0
3,9,0
4,10,1


In [24]:
#SMOTE==============================================================================
print("___________________________________________________________________\nSMOTE\n")
print('Original dataset shape %s' % Counter(y_train))
sm = SMOTE(sampling_strategy='float', ratio=0.5)
X_res, y_res = sm.fit_resample(X_train, y_train)
print('Resampled dataset shape %s' % Counter(y_res))

___________________________________________________________________
SMOTE

Original dataset shape Counter({0: 52738, 1: 12262})
Resampled dataset shape Counter({0: 52738, 1: 26369})


In [25]:
#CONSTRUCT DECISION TREE
clf = DecisionTreeClassifier()

#Hyperparameter tuning done for decision tree classifier
parameters={'min_samples_split' : range(10,100,10),'max_depth': range(1,20,2)}
clf_random = RandomizedSearchCV(clf,parameters,n_iter=15)
clf_random.fit(X_train, y_train)
grid_parm=clf_random.best_params_
print(grid_parm)

#Using the parameters obtained from HyperParameterTuning in the DecisionTreeClassifier 
clf = DecisionTreeClassifier(**grid_parm)
clf.fit(X_res,y_res)
clf_predict = clf.predict(X_test)
pred=pd.DataFrame(clf_predict,columns=["QuoteConversion_Flag"])


#Write into a file with actual prediction and corresponding probability
ID = test_data["QuoteNumber"]
pd.concat([ID,pred],axis=1).to_csv("/gdrive/My Drive/assignment3/DT1.csv", index = None)

res=pd.read_csv('/gdrive/My Drive/assignment3/DT1.csv')
res.head()






{'min_samples_split': 40, 'max_depth': 7}


,QuoteNumber,QuoteConversion_Flag
0,3,0
1,5,0
2,7,0
3,9,0
4,10,0


In [26]:
#Random Forest
rfc = RandomForestClassifier()
#Hyperparameter tuning for random forest
parameters={ 'n_estimators': range(50,150,20),'min_samples_split' : range(10,100,10),'max_depth': range(1,20,2)}
rfc_random = RandomizedSearchCV(rfc,parameters,n_iter=15)
rfc_random.fit(X_res, y_res)
grid_parm_rfc=rfc_random.best_params_
print(grid_parm_rfc)

rfc= RandomForestClassifier(**grid_parm_rfc)
rfc.fit(X_res,y_res)
rfc_predict = rfc.predict(X_test)
pred_rf=pd.DataFrame(rfc_predict,columns=["QuoteConversion_Flag"])

#Write into a file with actual prediction and corresponding probability
ID = test_data["QuoteNumber"]
pd.concat([ID,pred_rf],axis=1).to_csv("/gdrive/My Drive/assignment3/RF.csv", index = None)

res_rf=pd.read_csv('/gdrive/My Drive/assignment3/RF.csv')
res_rf.head()


{'n_estimators': 90, 'min_samples_split': 50, 'max_depth': 9}


,QuoteNumber,QuoteConversion_Flag
0,3,0
1,5,0
2,7,0
3,9,0
4,10,0


In [27]:
#SVM =====================================================================

start = time.time()
print(start)
svclassifier = LinearSVC()
svclassifier.fit(X_res, y_res)
y_pred = svclassifier.predict(X_test)
end = time.time()
print(end)

pred_svm=pd.DataFrame(y_pred,columns=["QuoteConversion_Flag"])

#Write into a file with actual prediction and corresponding probability
ID = test_data["QuoteNumber"]
pd.concat([ID,pred_svm],axis=1).to_csv("/gdrive/My Drive/assignment3/SVM.csv", index = None)

res_svm=pd.read_csv('/gdrive/My Drive/assignment3/SVM.csv')
res_svm.head()



1573605377.0971944
1573605490.0869884


,QuoteNumber,QuoteConversion_Flag
0,3,0
1,5,0
2,7,0
3,9,0
4,10,0


In [28]:
#KNN
from sklearn.neighbors import KNeighborsClassifier
knn_clf=KNeighborsClassifier(n_neighbors=5)
knn_clf.fit(X_res,y_res)
resDF = pd.DataFrame(knn_clf.predict(X_test), columns=["QuoteConversion_Flag"])

#Write into a file with actual prediction and corresponding probability
ID = test_data["QuoteNumber"]
pd.concat([ID,resDF],axis=1).to_csv("/gdrive/My Drive/assignment3/knn.csv", index = None)

res_knn=pd.read_csv('/gdrive/My Drive/assignment3/knn.csv')
res_knn.head()


,QuoteNumber,QuoteConversion_Flag
0,3,0
1,5,0
2,7,0
3,9,0
4,10,0


In [29]:
#MLP
mlp_clf=MLPClassifier(hidden_layer_sizes=(5,2))
mlp_clf.fit(X_res,y_res)
resMLP = pd.DataFrame(mlp_clf.predict(X_test), columns=["QuoteConversion_Flag"])

#Write into a file with actual prediction and corresponding probability
ID = test_data["QuoteNumber"]
pd.concat([ID,resMLP],axis=1).to_csv("/gdrive/My Drive/assignment3/mlp.csv", index = None)

res_mlp=pd.read_csv('/gdrive/My Drive/assignment3/mlp.csv')
res_mlp.head()

,QuoteNumber,QuoteConversion_Flag
0,3,0
1,5,0
2,7,0
3,9,0
4,10,0


In [30]:
#STACKING OF THE BEST THREE MODELS
print("___________________________________________________________________________________________\nEnsemble Methods Predictions using GradientBoosting, RandomForest and Decision Tree Classifier\n")
models = [rfc, clf, svclassifier]
S_Train, S_Test = stacking(models, 
                           X_res, y_res, X_test,
                           regression=False,
                           mode='oof_pred_bag', 
                           needs_proba=False,
                           save_dir=None,
                           metric=accuracy_score,
                           n_folds=4,
                           stratified=True,
                           shuffle=True,
                           random_state=0,
                           verbose=2)

___________________________________________________________________________________________
Ensemble Methods Predictions using GradientBoosting, RandomForest and Decision Tree Classifier

task:         [classification]
n_classes:    [2]
metric:       [accuracy_score]
mode:         [oof_pred_bag]
n_models:     [3]

model  0:     [RandomForestClassifier]
    fold  0:  [0.90924259]
    fold  1:  [0.90635587]
    fold  2:  [0.90200243]
    fold  3:  [0.90351942]
    ----
    MEAN:     [0.90528008] + [0.00277046]
    FULL:     [0.90528019]

model  1:     [DecisionTreeClassifier]
    fold  0:  [0.91642229]
    fold  1:  [0.91328311]
    fold  2:  [0.91292476]
    fold  3:  [0.90559264]
    ----
    MEAN:     [0.91205570] + [0.00397178]
    FULL:     [0.91205582]

model  2:     [LinearSVC]
    fold  0:  [0.70608757]
    fold  1:  [0.54598776]
    fold  2:  [0.67511125]
    fold  3:  [0.57802387]
    ----
    MEAN:     [0.62630261] + [0.06619901]
    FULL:     [0.62630361]



In [31]:
#RUNNING STACKING MODELS
#Random forest classifier
clf = DecisionTreeClassifier()

#Hyperparameter tuning for random forest
parameters={ 'min_samples_split' : range(5,50,10),'max_depth': range(2,10,2)}
rfc_random_s = RandomizedSearchCV(rfc,parameters,n_iter=10)
rfc_random_s.fit(S_Train, y_res)
grid_parm_rfc_s=rfc_random_s.best_params_
print(grid_parm_rfc_s)

rfc_stacking= RandomForestClassifier(**grid_parm_rfc_s)
rfc_stacking.fit(S_Train,y_res)
rfc_predict_s = rfc_stacking.predict(S_Test)
pred_rf_s=pd.DataFrame(rfc_predict_s,columns=["QuoteConversion_Flag"])

#Write into a file with actual prediction and corresponding probability
ID = test_data["QuoteNumber"]
pd.concat([ID,pred_rf_s],axis=1).to_csv("/gdrive/My Drive/assignment3/RF_s.csv", index = None)

res_rf_s=pd.read_csv('/gdrive/My Drive/assignment3/RF_s.csv')
res_rf_s.head()


{'min_samples_split': 15, 'max_depth': 6}


,QuoteNumber,QuoteConversion_Flag
0,3,0
1,5,0
2,7,0
3,9,0
4,10,0


In [32]:
#Decision Tree classifier
clf = DecisionTreeClassifier()
#Hyperparameter tuning done for decision tree classifier
parameters={'min_samples_split' : range(10,50,10),'max_depth': range(2,10,2)}
clf_random_s = RandomizedSearchCV(clf,parameters,n_iter=10)
clf_random_s.fit(S_Train, y_res)
grid_parm_s=clf_random_s.best_params_
print(grid_parm_s)

#Using the parameters obtained from HyperParameterTuning in the DecisionTreeClassifier 
clf_s = DecisionTreeClassifier(**grid_parm_s)
clf_s.fit(S_Train,y_res)
clf_predict_s = clf_s.predict(S_Test)
pred_s=pd.DataFrame(clf_predict_s,columns=["QuoteConversion_Flag"])


#Write into a file with actual prediction and corresponding probability
ID = test_data["QuoteNumber"]
pd.concat([ID,pred_s],axis=1).to_csv("/gdrive/My Drive/assignment3/DT1_s.csv", index = None)

res_s=pd.read_csv('/gdrive/My Drive/assignment3/DT1_s.csv')
res_s.head()


{'min_samples_split': 20, 'max_depth': 8}


,QuoteNumber,QuoteConversion_Flag
0,3,0
1,5,0
2,7,0
3,9,0
4,10,0


In [33]:
#GRADIENT BOOSTING 
model = GradientBoostingClassifier()
    
model = model.fit(S_Train, y_res)
y_pred_s = model.predict(S_Test)
pred_boost_s=pd.DataFrame(y_pred_s,columns=["QuoteConversion_Flag"])


#Write into a file with actual prediction and corresponding probability
ID = test_data["QuoteNumber"]
pd.concat([ID,pred_boost_s],axis=1).to_csv("/gdrive/My Drive/assignment3/boosting.csv", index = None)

res_boosting_s=pd.read_csv('/gdrive/My Drive/assignment3/boosting.csv')
res_boosting_s.head()


,QuoteNumber,QuoteConversion_Flag
0,3,0
1,5,0
2,7,0
3,9,0
4,10,0
